In [1]:
import numpy as np
import pandas as pd

import os

hasModel = False
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        if(filename == 'cardiovascular_model.pkl'):
            print('Already have cardiovascular_model.pkl')
            hasModel = True

Already have cardiovascular_model.pkl


In [2]:
df=pd.read_csv("./dataset/Cardiovascular.csv", delimiter=';')

In [3]:
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [4]:
df.describe()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
count,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000
mean,49972.419900,19468.865814,1.349571,164.359229,74.205690,128.817286,96.630414,1.366871,1.226457,0.088129,0.053771,0.803729,0.499700
std,28851.302323,2467.251667,0.476838,8.210126,14.395757,154.011419,188.472530,0.680250,0.572270,0.283484,0.225568,0.397179,0.500003
min,0.000000,10798.000000,1.000000,55.000000,10.000000,-150.000000,-70.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,25006.750000,17664.000000,1.000000,159.000000,65.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,50001.500000,19703.000000,1.000000,165.000000,72.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
75%,74889.250000,21327.000000,2.000000,170.000000,82.000000,140.000000,90.000000,2.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,99999.000000,23713.000000,2.000000,250.000000,200.000000,16020.000000,11000.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
df['BMI'] = df['weight'] / df['height'] ** 2
# df["BMI"]=df['BMI'].replace(0,df['BMI'].mean())
# df['Insulin']=df['Insulin'].replace(0,df['Insulin'].mean())
# df['SkinThickness']=df['SkinThickness'].replace(0,df['SkinThickness'].mean())
# df['BloodPressure']=df['BloodPressure'].replace(0,df['BloodPressure'].mean())
# df['Glucose']=df['Glucose'].replace(0,df['Glucose'].mean())
# df['DiabetesPedigreeFunction']=df['DiabetesPedigreeFunction'].replace(0,df['DiabetesPedigreeFunction'].mean())

In [6]:
df.isnull().sum()

id             0
age            0
gender         0
height         0
weight         0
ap_hi          0
ap_lo          0
cholesterol    0
gluc           0
smoke          0
alco           0
active         0
cardio         0
BMI            0
dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,confusion_matrix

import pickle

In [8]:
x=df[['age', 'gender', 'BMI', 'smoke']]
y=df['cardio']

In [9]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)

In [10]:
log_ref=LogisticRegression(random_state = 42)

In [11]:
log_ref.fit(x_train,y_train)

LogisticRegression(random_state=42)

In [12]:
parameters={
    'penalty':['l1', 'l2', 'elasticnet', None],
    'C':np.logspace(-3,3,7),
    'solver':['newton-cg','lbfgs','liblinear']
}
import warnings
warnings.filterwarnings('ignore')

In [13]:
if not hasModel:
    clf=GridSearchCV(
        log_ref,param_grid=parameters,
        scoring='accuracy',
        cv=10
    )
    clf.fit(x_train,y_train)
else:
    with open('cardiovascular_model.pkl','rb') as f:
        clf = pickle.load(f)

In [14]:
y_pred=clf.predict(x_test)

In [15]:
conf_mat=confusion_matrix(y_test,y_pred)
conf_mat

array([[5311, 3377],
       [3189, 5623]], dtype=int64)

In [16]:
accuracy=accuracy_score(y_test,y_pred)
accuracy

0.6248

In [17]:
with open('cardiovascular_model.pkl','wb') as f:
    pickle.dump(clf,f)

In [18]:
print('Successfully imported cardiovascular_model')

Successfully imported cardiovascular_model
